In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D

In [3]:
file_place = r"C:\Users\Conor\DataSets"

Breast_DF = pd.read_pickle(file_place + r"\Breast_data_raw")

In [4]:
Breast_DF.head()

912   \
Sex Type      Stage Pathology                 Grade Label Age Core             
F   Malignant IIa   Invasive ductal carcinoma 1     Blood 40  B4    0.094676   
                                                              B4    0.094676   
                                                              B4    0.095110   
                                                              B4    0.095979   
                                                              B4    0.097282   

                                                                        916   \
Sex Type      Stage Pathology                 Grade Label Age Core             
F   Malignant IIa   Invasive ductal carcinoma 1     Blood 40  B4    0.093373   
                                                              B4    0.093808   
                                                              B4    0.093808   
                                                              B4    0.095110   
                                                              B4    0.095979   

                                                                        920   \
Sex Type      Stage Pathology                 Grade Label Age Core             
F   Malignant IIa   Invasive ductal carcinoma 1     Blood 40  B4    0.095110   
                                                              B4    0.095110   
                                                              B4    0.095545   
                                                              B4    0.096848   
                                                              B4    0.097716   

                                                                        924   \
Sex Type      Stage Pathology                 Grade Label Age Core             
F   Malignant IIa   Invasive ductal carcinoma 1     Blood 40  B4    0.095545   
                                                              B4    0.095545   
                                                              B4    0.095979   
                                                              B4    0.097282   
                                                              B4    0.099019   

                                                                        928   \
Sex Type      Stage Pathology                 Grade Label Age Core             
F   Malignant IIa   Invasive ductal carcinoma 1     Blood 40  B4    0.096848   
                                                              B4    0.096848   
                                                              B4    0.097282   
                                                              B4    0.098151   
                                                              B4    0.099888   

                                                                        932   \
Sex Type      Stage Pathology                 Grade Label Age Core             
F   Malignant IIa   Invasive ductal carcinoma 1     Blood 40  B4    0.097282   
                                                              B4    0.097282   
                                                              B4    0.097716   
                                                              B4    0.099019   
                                                              B4    0.100322   

                                                                        936   \
Sex Type      Stage Pathology                 Grade Label Age Core             
F   Malignant IIa   Invasive ductal carcinoma 1     Blood 40  B4    0.095979   
                                                              B4    0.096848   
                                                              B4    0.096848   
                                                              B4    0.097716   
                                                              B4    0.099019   

                                                                        940   \
Sex Type      Stage Pathology                 Grade Label Age Cor

Create the D_Matrix by first encoding the desired categorical labels to numbers.

# Preprocessing

In [5]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, RobustScaler

steps = [
         #("Normalise", Normalizer(norm = "l1"))
         ("RobustScaler", RobustScaler())
        ]

pipe  = Pipeline(steps)

C:\Users\Conor\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Conor\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Conor\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Conor\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Conor\Anaconda3\lib\site-packages\scipy\_lib\_numpy_compat.py:10: DeprecationWarning: Importing from numpy.testing.nosetester is deprecated since 1.15.0, import from numpy.testing instead.
  from numpy.testing.nosetester import import_

In [6]:
import Tissue_Analysis_Tools as TAT

PCA_NR_Components = 50

dataframe = TAT.process_data(Breast_DF, paraffin = (1340,1490), balance = "Type")

PCA_reduced = TAT.clean_spectra(dataframe, PCA_NR_Components)

values = pipe.fit_transform(PCA_reduced)

Breast_DF_P = pd.DataFrame(values, index = dataframe.index, columns = dataframe.columns)

C:\Users\Conor\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Conor\AppData\Roaming\Python\Python36\site-packages\pandas\core\frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# Analysis

In [ ]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

le = LabelEncoder()
lb = LabelBinarizer()

DATA = Breast_DF_P#.sample(1000)

X = DATA
Y = DATA.reset_index()["Type"]
Y_b = lb.fit_transform(Y)

In [ ]:
import xgboost as xgb

In [ ]:
from scipy import stats

param_dist = {'n_estimators': stats.randint(50, 500),
              'learning_rate': stats.uniform(0.01, 0.07),
              'subsample': stats.uniform(0.3, 0.7),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.45),
              'min_child_weight': [1, 2, 3]
             }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GroupKFold, GroupShuffleSplit, LeavePGroupsOut

In [ ]:
Breast_DF_P.reset_index()["Type"].value_counts()

In [ ]:
list(enumerate(np.unique(Y)))

In [ ]:
from sklearn.metrics import make_scorer, roc_auc_score, precision_score, recall_score, auc

scorers = {
    # "ROC_AUC": make_scorer(roc_auc_score),
    "Precision": make_scorer(precision_score)
    ,"Sensitivity": make_scorer(recall_score)
    #,"AUC": make_scorer(auc)
          }

In [ ]:
% time

n_Folds = 2

results = dict()

groups =  DATA.reset_index()["Core"].values

print(len(groups))

for i, name in enumerate(np.unique(Y)):
    
    # Find out the ratio of positive classes to negative to scale positive weights.
    # Allows for compensation of unbalanced classes without throwing away data.
    
    pos_ratio = 1/(np.sum(Y_b[:,i])/Y_b[:,i].shape[0])
    
    print(np.unique(Y_b[:,i]))

    clf_xgb = xgb.XGBClassifier(objective = "binary:logistic", n_classes = 2, n_jobs = -1, scale_pos_weight = pos_ratio)

    clf = RandomizedSearchCV(clf_xgb, param_distributions = param_dist
                             , n_iter = 10, scoring = scorers, refit = "Precision"
                             , error_score = 0, verbose = 3
                             , n_jobs = -1, return_train_score = True
                             #, cv = GroupKFold(n_Folds).split(X, Y_b[:,i], DATA.reset_index()["Core"])
                             #, cv = GroupShuffleSplit(n_Folds, train_size = 0.3).split(X, Y, DATA.reset_index()["Core"])
                             , cv = LeavePGroupsOut(50).split(X, Y_b[:,i], groups)
                            )
    
    results[name] = clf.fit(X,Y_b[:,i])

In [ ]:
results["Normal"].cv_results_

In [ ]:
df = pd.DataFrame(results["Normal"].cv_results_)

plt.clf()
threedee = plt.figure().gca(projection='3d')
threedee.scatter(df["param_colsample_bytree"], df["param_learning_rate"], df["mean_test_Precision"])
plt.show()

In [ ]:
for key, value in results.items():
    
    print(key, "\n")
    print(dir(value.cv_results_))